### Let's Understand About the Tokenizer

In [1]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

🔥 Conclusion

✔ input_ids → Tokens ka numerical representation

✔ attention_mask → Padding ignore karne ka indicator

✔ token_type_ids → Batata hai kaunsa sentence pehla hai aur kaunsa doosra

 (BERT ke NSP task ke liye zaroori)

In [11]:
sequence = "Using a Transformer network is simple"

In [12]:
tokenizer(sequence)

{'input_ids': [101, 2478, 1037, 10938, 2121, 2897, 2003, 3722, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [3]:
sequence1 = "Using a Transformer network is simple"

In [13]:
tokenizer(sequence1)

{'input_ids': [101, 2478, 1037, 10938, 2121, 2897, 2003, 3722, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [4]:
sequence2 = "It is very powerful"

In [14]:
tokenizer(sequence2)

{'input_ids': [101, 2009, 2003, 2200, 3928, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [5]:
res = tokenizer(sequence1,sequence2)
print(res)

{'input_ids': [101, 2478, 1037, 10938, 2121, 2897, 2003, 3722, 102, 2009, 2003, 2200, 3928, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [6]:
tokens = tokenizer.tokenize(sequence)
print(tokens)


['using', 'a', 'transform', '##er', 'network', 'is', 'simple']


In [7]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[2478, 1037, 10938, 2121, 2897, 2003, 3722]


In [8]:
decoded_string = tokenizer.decode(ids)
print(decoded_string)

using a transformer network is simple


Raw Text → Tokens → Token IDs → Model Input (Embeddings, Attention Mask, etc.)

In [9]:
label_mapping = {
    "O": 0,        # No entity
    "B-PER": 1,    # Beginning of a Person Name
    "I-PER": 2,    # Inside a Person Name
    "B-ORG": 3,    # Beginning of an Organization Name
    "B-LOC": 4,    # Beginning of a Location Name
}

In [15]:
sentences = ["Hello Sunny", "I love Apple"]
# [0, 1] [0, 0, 3]

In [16]:
labels = [[0, 1], [0, 0, 3]]  # Different length labels

In [17]:
# Tokenizing with padding
tokenized_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

In [18]:
tokenized_inputs

{'input_ids': tensor([[  101,  7592, 11559,   102,     0],
        [  101,  1045,  2293,  6207,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1]])}

In [19]:
tokenized_inputs["input_ids"]

tensor([[  101,  7592, 11559,   102,     0],
        [  101,  1045,  2293,  6207,   102]])

In [20]:
# Manually padding labels
max_length = tokenized_inputs["input_ids"].shape[1]

In [21]:
max_length

5

In [22]:
[l for l in labels]

[[0, 1], [0, 0, 3]]

In [23]:
l=[0, 1]

In [24]:
len(l)

2

In [25]:
max_length

5

In [26]:
l + [-100] * (max_length - len(l))

[0, 1, -100, -100, -100]

In [27]:
padded_labels = [l + [-100] * (max_length - len(l)) for l in labels]

In [28]:
padded_labels

[[0, 1, -100, -100, -100], [0, 0, 3, -100, -100]]

In [29]:
print("Tokenized Inputs:", tokenized_inputs["input_ids"])
print("Padded Labels:", padded_labels)

Tokenized Inputs: tensor([[  101,  7592, 11559,   102,     0],
        [  101,  1045,  2293,  6207,   102]])
Padded Labels: [[0, 1, -100, -100, -100], [0, 0, 3, -100, -100]]


In [31]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


In [32]:
tokenizer("Hello Sunny")["input_ids"]

[101, 7592, 11559, 102]

In [33]:
tokenizer("I love Apple")["input_ids"]

[101, 1045, 2293, 6207, 102]

In [34]:
[[0, 1], [0, 0, 3]]

[[0, 1], [0, 0, 3]]

In [35]:
batch = [
    {"input_ids": tokenizer("Hello Sunny")["input_ids"], "labels": [0, 1]},
    {"input_ids": tokenizer("I love Apple")["input_ids"], "labels": [0, 0, 3]},
]


In [36]:
collated_batch = data_collator(batch)

In [37]:
print("Collated Input IDs:", collated_batch["input_ids"])
print("Collated Labels:", collated_batch["labels"])

Collated Input IDs: tensor([[  101,  7592, 11559,   102,     0],
        [  101,  1045,  2293,  6207,   102]])
Collated Labels: tensor([[   0,    1, -100, -100, -100],
        [   0,    0,    3, -100, -100]])
